<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_09/blob/Modifications-to-Q2/test_2_v9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation  # Need to import extra functionality for animations

# These are the inputs that act as parameters that allow the user to specify model domain, resolution, and boundary conditions at the edge of the domain
end_time = float(input("How long should the model last for, in seconds  "))
time_interval = float(input("The time interval, in seconds   "))
length = float(input("Length of model, in metres   "))
length_interval = float(input("The length interval, in metres   "))
speed = float(input("Speed of river flow, in m/s   "))

# Read the CSV file
csv_data = pd.read_csv('initial_conditions.csv', encoding='cp1252')

# Extract the distance and concentration columns from the CSV
csv_distance = csv_data.iloc[:, 0].values.astype(float)
csv_concentration = csv_data.iloc[:, 1].values.astype(float)

# INTERPOLATION: Use numpy's interp function to interpolate CSV data onto model grid
initial_conditions = np.interp(distance, csv_distance, csv_concentration)

# Print interpolation info for clarity
print(f"\nInterpolation complete:")
print(f"  CSV data points: {len(csv_distance)} (at {csv_distance[1] - csv_distance[0]:.1f}m intervals)")
print(f"  Model grid points: {len(distance)} (at {length_interval}m intervals)")
print(f"  Initial concentration at x=0: {initial_conditions[0]:.1f} μg/m³")
print(f"  Initial concentration at x={length}m: {initial_conditions[-1]:.1f} μg/m³")

# Get boundary condition (concentration at x=0) from the interpolated data
concentration_at_x0 = initial_conditions[0]

# The A and B values simplify the advection equation calculation
A_value = 1 / time_interval + speed / length_interval
B_value = speed / length_interval

# Calculate number of frames (time steps)
num_frames = int(end_time / time_interval) + 1

# Pre-calculate all concentration arrays for each time step
all_concentrations = [initial_conditions.copy()]
current_concentration = initial_conditions.copy()

for k in range(1, num_frames):
    new_concentration = np.zeros(len(distance))
    new_concentration[0] = concentration_at_x0

    for i in range(1, len(distance)):
        new_concentration[i] = (current_concentration[i] / time_interval + B_value * new_concentration[i - 1]) / A_value

    all_concentrations.append(new_concentration.copy())
    current_concentration = new_concentration.copy()

# Set some attributes of the animation object
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()  # interactive off

# Create your blank figure
fig, ax = plt.subplots()

# Make your function with the frame number as the argument
def animate(f):
    plt.cla()  # Make sure you clear the figure at the start of each iteration

    # Create your new data to be plotted based off of the frame
    concentration = all_concentrations[f]

    # Draw the plot for this frame
    plt.plot(distance, concentration)

    # Set axis limits and labels
    plt.xlim(0, length)
    plt.ylim(0, concentration_at_x0 * 1.1)
    plt.xlabel("Distance (m)")
    plt.ylabel("Concentration (μg/m³)")
    plt.title(f"Concentration at Time = {f * time_interval:.1f} s")
    plt.grid(True)

# Create the animation
matplotlib.animation.FuncAnimation(fig, animate, frames=num_frames)